# Introduction to Bayesian Networks (BNs)



## CSCI E-83
## Stephen Elston

**Bayesian Networks** or **BNs** are one of the basic representations of probabilistic graphical models. BNs are **directed acyclic graphs** or **DAGs** which form a compact representation of a joint distribution. 

The representation captures the **conditional independence structure** of the joint distribution. Taking advantage of the independency structure of the distribution can greatly reduce the computational complexity of inference on the graphic model. 

The directed nature of the BN can capture **causality**. **Influence** travels along the directed edges of the graph. However, keep in mind, information on independence can flow both directions along the graph.

The **conditional marginal distribution** of each variable is only dependent on its **parents**. In other words, **belief** is propagated along the DAG. In this sense, the DAG is a representation of a **generative sampling process**. 

Graphical models allow us to add **evidence** as data is observed for certain variables. Further, we can perform **queries** on **hidden variables**, variables with no evidence. 

## Some graph terminology

Before we dive into the probabilistic graphical models let's examine some basic graph terminology. Graphs are are comprised of two types of elements:
1. **Nodes** or **vertices** in probabilistic models contain the distribution information.
2. **Edges** define the relationships between the nodes. Edges can be **directed** or **undirected**. Information flows in the direction specified along a directed edge. Undirected edges have no preferred direction of information flow. 
3. The set of edges if a graph (less nodes) is called the **skeleton**. 
4. Nodes connected by an edge are **neighbors**. 
5. The **degree** of a node is the number of neighbors. 
6. **Parents** are nodes with a directed edge to another node, know as the **child**. Parents are said to have **influence** on their children. 
7. **Ancestors** are nodes which are grandparents, great grandparents, etc, of a with a path of directed edges to a child. 
8. **Descendants** are nodes which receive information from parents or ancestors along a path of directed edges. For example, a child node is a descendant of its parents and grand parents. 
9. **Leaf** nodes have no directed edges to ancestors. 

We can classify graphs by their topology. The figure below shows some basic graph types.   

<img src="img/GraphTypes.JPG" alt="Drawing" style="width:600px; height:200px"/>
<center> **Common ** </center>

Four graph types are illustrated in the figure above:   
1. **Directed acyclic graph** is a graph where all edges are **directed** and there are no **cycles**. Information or **influence** flows in one direction along a directed edge. In the case illustrated, influence flows  $A \rightarrow B$ and $C \rightarrow B$, but not the other way. This makes $A$ and $C$ **ancestors** or **parents** of $B$. An **acyclic graph** has no cycle.
2. **Directed cyclic graph** has directed edges but contains a cycle. A **cycle** is a directed path that starts and returns to the same node. In this case the illustrated case the cycle is $A \rightarrow B \rightarrow C \rightarrow A$.
3. **Undirected graph** has no directed edges. 
4. **Partially directed graph** has both directed and undirected edges. 


## A simple example

We have already worked on the student GRE score and letter problem. As a reminder, a student would like to make inferences or **queries** on the joint distribution $P(D,I,S,G,L)$. She can make the following assertions about the independencies for this problem: 

1. The degree of difficulty, D, of the machine learning course is **unconditionally independent** of all other variable, $\{D \bot I,S,G.L \}$.
2. The intelligence of the student is, I, **unconditionally independent** of all other variable, $\{I \bot D,S,G.L \}$.
3. The quality of the student's recommendation letter, L, is **conditionally independent**  of intelligence, her GRE score, and her letter, given her grade, $\{L \bot I,S,L\ |\ G\}$.
4. The student's grade in the machine learning course, G, is **conditionally independent** of her GRE score and her letter, give the difficulty of the course and her intelligence, $\{G  \bot S,L\ |\ I,D\}$. 
5. The students GRE scores, S, are **conditionally independent** of her grade, difficulty of the course and letter, given her intelligence, $\{S \bot G,D,L\ |\ I\}$.

It is readily possible to construct a BN that represents the dependencies on this distribution as shown in the figure below.  

<img src="img/LetterDAG.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center> **DAG for the student score and letter distribution** </center>


## Using pgmpy for directed graphical model

We can use the Python pgmpy package to create and manipulate Bayesian DAGs. You can find complete documentation on this package [here](http://pgmpy.org/).

If you have not installed pgmpy, un-comment the code in the cell below and execute it.  

In [ ]:
#!pip install pgmpy

To import the parts of pgmpy we will use for the examples in this notebook, execute the code in the cell below:

In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

Continuing with the example, we need to define the graph. The first step is to define the **skeleton** of the graph. This is done by defining the edges. Each edge is define as a tuple; (start node, end node). Execute the code in the cell below to create the skeleton of the graph      

In [2]:
## Define the network structure.
## The first value of the tuple defines the origin
## of the connector and the second the terminal point
## of the directed edge. 
student_model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

With the graph skeleton defined, we need to specify the probability distributions of the nodes. In this case, we are using discrete distributions.   

We will start with the two independent variables, $D$ and $I$, which are binomially distributed. Notice that the sum of the probabilities must add to 1.0.

In [3]:
## Define the independent variables
CPD_D = TabularCPD(variable='D', variable_card=2, values=[[0.7, 0.3]])
print(CPD_D)
CPD_I = TabularCPD(variable='I', variable_card=2, values=[[0.8, 0.2]])
print(CPD_I)

╒═════╤═════╕
│ D_0 │ 0.7 │
├─────┼─────┤
│ D_1 │ 0.3 │
╘═════╧═════╛
╒═════╤═════╕
│ I_0 │ 0.8 │
├─────┼─────┤
│ I_1 │ 0.2 │
╘═════╧═════╛


Next, we will deal with the children of the two independent variables. Now we need to define the variables associated with **evidence variables**. An evidence variable is an **observable** variable for which data can be acquired. We will explore observable variables and evidence later. 

The letter variable, $L$, is 

In [4]:
# Define the distributions with a single conditional variable 
# or evidence. 
CPD_L = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'], # Leter depends on the grade
                   evidence_card=[3])
print(CPD_L)

CPD_S = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'], # GRE score depneds on intelligence
                   evidence_card=[2])
print(CPD_S)

╒═════╤═════╤═════╤══════╕
│ G   │ G_0 │ G_1 │ G_2  │
├─────┼─────┼─────┼──────┤
│ L_0 │ 0.1 │ 0.4 │ 0.99 │
├─────┼─────┼─────┼──────┤
│ L_1 │ 0.9 │ 0.6 │ 0.01 │
╘═════╧═════╧═════╧══════╛
╒═════╤══════╤═════╕
│ I   │ I_0  │ I_1 │
├─────┼──────┼─────┤
│ S_0 │ 0.95 │ 0.2 │
├─────┼──────┼─────┤
│ S_1 │ 0.05 │ 0.8 │
╘═════╧══════╧═════╛


In [5]:
# Define the distributions with a multiple conditional variables 
# or evidence. 
CPD_G = TabularCPD(variable='G', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])
print(CPD_G)

╒═════╤═════╤══════╤══════╤═════╕
│ I   │ I_0 │ I_0  │ I_1  │ I_1 │
├─────┼─────┼──────┼──────┼─────┤
│ D   │ D_0 │ D_1  │ D_0  │ D_1 │
├─────┼─────┼──────┼──────┼─────┤
│ G_0 │ 0.3 │ 0.05 │ 0.9  │ 0.5 │
├─────┼─────┼──────┼──────┼─────┤
│ G_1 │ 0.4 │ 0.25 │ 0.08 │ 0.3 │
├─────┼─────┼──────┼──────┼─────┤
│ G_2 │ 0.3 │ 0.7  │ 0.02 │ 0.2 │
╘═════╧═════╧══════╧══════╧═════╛


## Independency and uniqueness of BNs

We have just defined a BN with the Independency structure of the distribution defined by the assertions. The independency structure is quite important since by imposing independencies can greatly reduce the computational complexity of the graph.   

However, there is no guarantee that a given BN uniquely defines the independency structure of a distribution. A simple example of four BNs representing the same independency structure is shown in the figure below. In each case, the following **conditional independence** assertions are true:

$$A\ \bot\ C\ |\ B\\
Or,\\
C\ \bot\ A\ |\ B$$


<img src="img/Dependency.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center> **Multiple BNs with same dependence structure** </center>

Each of these cases supports that   

1. **Causal or Cascade:** In this case A causes B which causes C. This information is represented in the directed edges. If B is an evidence variable then A and C are separated and cannot dependent. Once a value is assigned to B, A depends on B and C depends on B, but A and C are **decoupled**. 
2. **Evidential:** Once again, the evidence variable B separates A and C, just as they do in the causal case. In this case, evidence is added to the DAG which allows inference of A.   
3. **Common Evidence, V-Structure, or Collider:** In this case, having evidence on B **blocks** the path or **separates** A from C. In other words, knowing B explains away A and C. 
4. **Common Cause or Common Parent:** In this case, B is causal to both A and C, making A and C independent. 

A key point here is that any of these DAGs has the same aforementioned independence properties. In other words, multiple DAGs can exhibit the same independencies. We has that these DAGs exhibit **I-equivalence**, since they have the same independencies. 

We can write a generalization of the independence properties of a BN:

> **Definition:** On a graph $G$, the variable $X_i$ is independent of its **nondecendents** given its **parents**, $Pa_{X_i}$. We can say that $G$ includes as set of **local conditional independence assumptions**:

$$I_{\ell}(G): \{X_i\ \bot\ Nondecendants_{X_i}\ |\ Pa_{X_i}:\ \forall i\ \}$$

Let's look at some more examples of applying this definition The Figure below illustrates some key cases.

<img src="img/Independencies2.JPG" alt="Drawing" style="width:600px; height:350px"/>
<center> **Multiple BNs with same dependence structure** </center>

Following the numbering of each DAG, we can make make the following assertions (statements) about independencies in each of these DAGs:

1. **$P(A,B,C) = P(B\ \| A,C)\ P(A)\ P(C)$:** In this case, A and C are causes of B, and are therefore independent.
2. **$P(A,B,C) = P(A\ \| B)\ P(\ C |\ B)\ P(B)$:** Here, B is the cause of independent effects A and C.
3. **$P(A,C\ \| B) = P(A\ \| B)\ P(\ C |\ B)$:** A and C are independent given B. While the causes of A and C are a-priori independent, having evidence for B tells us about the causes A and C. This is the same case illustrated in the previous figure. 
4. **$P(A,C\ \| B) = P(A\ \| B)\ P(\ C |\ B)$:** A and C are independent when conditioned on B. Given the cause B we know everything about each effect A and C. 
5. **$P(A,C\ \| B) = P(A\ \| B)\ P(\ C |\ B)$:** Since D is a descendant of B, this case is the same as 3 above.
6. **$P(A,C) = P(A)\ P(C)$:** A and C are made independent by marginalizing over B. In other works, in the absence of specific information on B, A and C are unconditionally independent.
7. **$P(A,C) \ne P(A)\ P(C)$:** In this case, A and C are dependent when marginalizing over B. In other words the effects, A and C are dependent on B. 

Keeping the above in mind, we can find the independencies of the DAG representing the distribution for the student letter and GRE score problem:

In [6]:
## Find the local independencies in the BN
# Getting all the local independencies in the network.
student_model.local_independencies(['D', 'I', 'S', 'G', 'L'])

(D _|_ G, L, I, S)
(I _|_ G, D, L, S)
(S _|_ G, D, L | I)
(G _|_ S, L | D, I)
(L _|_ S, D, I | G)

## DAGs and independence maps

We can generalize beyond the above example we can say that a DAG represents a probability distribution **consistent** with the DAG. We can factor the the graph:
$$P(X) = \prod_{i=1:d}P(x_i|X_{\pi_i})$$

Where, $P(X)$ is probability distribution, $X_{\pi_i}$ is the set of **parents** of $x_i$, a node on the graph, and $d$. In other words, in a DAG a node $x_i$ is independent of all other nodes give the set of the node's parents, $X_{\pi_i}$.

We can also generalize the idea of independencies with the **local Markov assumption** applied to a DAG G by writing:

$$I(G) = \{ X \bot Z\ |\ Y:\ dsep_G(X:Z|Y) \} $$ 

This leads us to an important definition of the properties of DAGs:

> **Definition:** A DAG, G, is an **independence map** or **I-map** of a distribution P if $I_l(G) \subseteq I(P)$, where $I(P)$ is the set of independencies of the distribution P and $I_l(G)$ is the set of independencies of the DAG. We can express this relationship as:

$$(X\ \bot\ Y\ |\ Z_G) \Rightarrow (X\ \bot\ Y\ |\ Z_P)$$

We can see that there are multiple DAGs for which $I_l(G) \subseteq I(P)$ can be true.  But, what are the practical implications of this definition? 

From the above definition, you can see that it can be the case that a DAG can be an I-map, but not a complete representation of the independencies of $P$. This leads us to another definition:

>**Definition:** A DAG, G, is a **minimal I-map** for a distribution P if removal of even a single edge renders $G$ not an I-map. 

The concept of an I-map is quite useful. In fact, no graph $G$ can be considered a useful representation of a distribution $P$ if it is not an I-map of $P$. A minimal I-map will have the minimum possible complexity, but may be hard to find in practice. 

Note that a minimal I-map may not be unique for a distribution $P$. If two DAGs have the same I-map, we say they are **I-equivalent**. 

## Trails in DAGs


The concept of **trails** provides another way to think about separation in DAGs. You can think of a trail as a path between one subset of a DAG and another subset of the DAG. If the trail is **active** there is no independency between the  If a trail between subsets is **blocked** the subsets are independent. Trails are blocked by evidence or observed. Following on the previous figure we can define four cases:

1. **Causal trail;** is active if and only if B is not observed. 
2. **Evidential trail;** is active if and only if B is not observed.
3. **Common cause;** is active if and only if B is not observed.
4. **Common effect;** is active if and only if B or one of B's decendants is observed.  

The independencies resulting from trails can be generalized s **d-separation**:

> **Definition:** Let $X$, $Y$ and $Z$ be three subsets of graph G. We say $X$ and $Y$ are **d-separated** given Z if there is no active trail between and node $x \in X$ and any othee node $y \in Y$ given Z. We can express d-separation as $d-sep_G(X;Y\ |\ Z)$.   

The intuition is that when there is evidence on the trail between subsets, the subset with evidence effectively blocks the trail creating the separation.   

In [7]:
student_model.is_active_trail('D','G')

True

## Evidence, hidden variables and queries

Up to now we have only dealt with distributions of the variables in our graphical models. 

It is possible that we will **observe** data values for some variables, which is typically referred to as **evidence**. Evidence provides specific values for **observable variables**. Once we have evidence, the size of the tables we must work with can be reduced. Only entries consistent with the evidence need be retained. Thus, computational complexity can be reduced, sometimes greatly. 

An example of evidence might be a specific score $S$ on the GRE test for a student. If the score is high, $S^1$, then we need not consider table entries with a low GRE score $S^0$.

Not all variables are observable however. We must estimate the marginal distribution of these **unobservable variables** given the evidence. We say that we perform a **query** on the unobservable variables. The query returns the marginal distribution of the variable. 

For example, human intelligence cannot be directly observed. In our student model, we might observe a student's GRE score as $S^1$ and course grade as $G^0$. Given this evidence, we can now query to compute the marginal posterior distribution of the student's intelligence. 



## Uncertain and unreliable evidence

In the real world, evidence can be **uncertain** or **unreliable**. Both of these situations effect the marginal distribution returned by a query on any unobservable variable. 

### Uncertain evidence

First, let's consider the effect of **uncertain evidence**. There are many situations in which uncertain evidence can arise. For example, an observer might only have a certain confidence in an observation. An ornithologist sees a bird, but may only be 70% certain of the species. The quality of the habitat (an unobservable) is a parent of species present. How can we include the uncertainty of the ornithologist's observation into the model? 

A model of habitat quality, $q$, given the species, $s$ can be written:

$$p(q\ |\ s) = \sum_s p(q,s\ |\ \tilde{s}) = \sum_s p(q\ |\ s, \tilde{s})\ p(s\ |\ \tilde{s})$$ 

Now, $q$ is dependent on $s$ independent of the uncertainty of the observation $\tilde{s}$, leading to:

$$\sum_s p(q\ |\ s)\ p(s\ |\ \tilde{s})$$

### Unreliable evidence

Let's say our ornithologist decides her observations are going too slowly and decides to hire an assistant. Unfortunately, the person she hires has exaggerated his sill at bird species identification. 

## Knowledge engineering with DAGs

We have seen how DAGs are a powerful and compact representation of joint distributions. But, how do we specify the DAG? There are two distinct aspects of the specification process:

1. The **qualitative specification** which defines the skeleton of the DAG. The skeleton specifies the influences in the belief network. There are a number of ways the qualitative specification of a DAG including: 
  - **Prior knowledge** of influence or causal relationships based on knowledge of the probability distributions in the domain. 
  - **Eliciting assessments of experts** as to which variables are influential or causal to other variables. 
  - **Learning** the independence structure of the probability distribution from data. This is a computationally difficult problem involving .      
      
      
2. The **quantitative specification** which defines the details of the conditional distributions. There are a number of ways to create the quantitative specification of a DAG: 
  - **Prior knowledge** of the conditional distributions.
  - **Empirically determining the conditional distributions** from data. 
  - **Eliciting opinions of experts** as to the likelihood of events. 
  - **Fitting** parameters of a distribution model.    
     
  
It should be noted that multiple methods are often used to create these specifications.   

#### Copyright 2018, Stephen F Elston. All rights reserved.